# **KPMG Data Analytics Virtual Internship**

## Task 2
### Data Insights

    Targeting high value customers based on customer demographics and attributes.

#### Here is our task
For context, Sprocket Central Pty Ltd is a long-standing KPMG client whom specialises in high-quality bikes and accessible cycling accessories to riders. Their marketing team is looking to boost business by analysing their existing customer dataset to determine customer trends and behaviour. 

Using the existing 3 datasets (Customer demographic, customer address and transactions) as a labelled dataset, please recommend which of these 1000 new customers should be targeted to drive the most value for the organisation. 

In building this recommendation, we need to start with a PowerPoint presentation which outlines the approach which we will be taking. The client has agreed on a 3 week scope with the following 3 phases as follows - Data Exploration, Model Development, and Interpretation.

Prepare a detailed approach for completing the analysis including activities – i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off. Please advise what steps you would take. 

Please ensure your PowerPoint presentation includes a detailed approach for our strategy behind each of the 3 phases including activities involved in each - i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off.

# Data Exploration

## Import Library

In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('Matplotlib', mpl.__version__)
print('Seaborn', sns.__version__)

NumPy 1.24.2
Pandas 2.0.1
Matplotlib 3.7.1
Seaborn 0.12.2


## Read Dataset

In [ ]:
# Read the Transactions dataset
txn = pd.read_excel('Transactions.xlsx')

# Read the Customer List dataset
old_c = pd.read_excel('OldCustomerList.xlsx')
new_c = pd.read_excel('NewCustomerList.xlsx')
all_c = pd.read_excel('AllCustomerList.xlsx')

## 0. Transactions Dataset

In [ ]:
# Dimensions of the DataFrame
txn.shape

In [ ]:
print('Total rows: {}'.format(txn.shape[0]))
print('Total cols: {}'.format(txn.shape[1]))

In [ ]:
# Data types of the columns
txn.info()

In [ ]:
# Display a few rows
txn.head()

In [ ]:
txn = txn.sort_values('customer_id')
txn.head()

In [ ]:
txn[txn['last_purchases'] > 365].shape

In [ ]:
rfmTable = txn.groupby('customer_id').agg({
    'last_purchases': lambda x: x.min(),
    'customer_id': lambda x: len(x),
    'profit': lambda x: x.sum()
})

rfmTable.rename(columns={
    'last_purchases': 'recency', 
    'customer_id': 'frequency', 
    'profit': 'monetary_value'
    }, inplace=True
)

rfmTable.head()

In [ ]:
quartiles = rfmTable.quantile(q=[0.25,0.50,0.75])
quartiles

In [ ]:
# Define functions of RFM OneHotEncoder
def r_OHE(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.5]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else: 
        return 4
    
def fm_OHE(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.5]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else: 
        return 1

In [ ]:
rfmSegment = rfmTable
rfmSegment['r_score'] = rfmSegment['recency'].apply(r_OHE, args=('recency', quartiles))
rfmSegment['f_score'] = rfmSegment['frequency'].apply(fm_OHE, args=('frequency', quartiles))
rfmSegment['m_score'] = rfmSegment['monetary_value'].apply(fm_OHE, args=('monetary_value', quartiles))
rfmSegment.head()

In [ ]:
rfmSegment['rfm_class'] = 100 * rfmSegment['r_score'] + 10 * rfmSegment['f_score'] + rfmSegment['m_score']
rfmSegment['total_score'] = rfmSegment['r_score'] + rfmSegment['f_score'] + rfmSegment['m_score']
rfmSegment.head()

In [ ]:
rfm_quartiles = rfmSegment['rfm_class'].quantile(q=[0, 0.25, 0.5, 0.75, 1]).values
rfm_quartiles

In [ ]:
# Define function of RFM Class OneHotEncoder
def rfm_class_OHE(x, p, d):
    if x <= d[0]:
        return 'Platinum'
    elif x <= d[1]:
        return 'Gold'
    elif x <= d[2]: 
        return 'Silver'
    else: 
        return 'Classic'

rfmSegment['customer_level'] = rfmSegment['rfm_class'].apply(rfm_class_OHE, args=('rfm_class', rfm_quartiles))

In [ ]:
rfmSegment.info()

In [ ]:
rfmSegment.head()

## 1. OldCustomerList Dataset

### 1.0. Merge Old Customer List to RFM Segment

In [ ]:
old_c.columns

In [ ]:
old_c = old_c.drop(['first_name', 'last_name',
                    'past_3_years_bike_related_purchases',
                    'DOB', 'job_title', 'deceased_indicator',
                    'tenure', 'address', 'postcode', 'country',
                    'property_valuation'], axis=1)

In [ ]:
old_c_txn = pd.merge(rfmSegment, old_c, left_index=True, right_index=True)
old_c_txn.columns

In [ ]:
old_column_order = ['customer_id', 'age', 'gender', 'job_industry_category', 'wealth_segment',
                    'owns_car', 'state', 'recency', 'frequency', 'monetary_value',
                    'r_score', 'f_score', 'm_score', 'rfm_class', 'total_score',
                    'customer_level']

# Reorder column
old_c_txn = old_c_txn.reindex(columns=old_column_order)
old_c_txn.columns

In [ ]:
old_c_txn.shape

In [ ]:
old_c_txn.head()

### 1.1. Data Insights

#### 1.1.0. Recency, Frequency, and Monetary

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3))

old_c_txn.groupby('r_score').agg('recency').count().to_frame().plot(kind='bar', ax=axs[0])
axs[0].set_title('Recency')
axs[0].legend(loc='lower center')

old_c_txn.groupby('f_score').agg('frequency').count().to_frame().plot(kind='bar', ax=axs[1])
axs[1].set_title('Frequency')
axs[1].legend(loc='lower center')

old_c_txn.groupby('m_score').agg('monetary_value').count().to_frame().plot(kind='bar', ax=axs[2])
axs[2].set_title('Monetary')
axs[2].legend(loc='lower center')

plt.tight_layout(pad=2)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3))

old_c_txn.groupby('total_score').agg('recency').mean().to_frame().plot(kind='bar', ax=axs[0])
axs[0].set_title('Recency')

old_c_txn.groupby('total_score').agg('frequency').mean().to_frame().plot(kind='bar', ax=axs[1])
axs[1].set_title('Frequency')

old_c_txn.groupby('total_score').agg('monetary_value').mean().to_frame().plot(kind='bar', ax=axs[2])
axs[2].set_title('Monetary')

plt.tight_layout(pad=2)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8,3))

sns.scatterplot(data=old_c_txn, x='recency', y='monetary_value', ax=axs[0])
axs[0].set_title('Recency vs. Monetary')

sns.scatterplot(data=old_c_txn, x='recency', y='frequency', ax=axs[1])
axs[1].set_title('Recency vs. Frequency')

sns.scatterplot(data=old_c_txn, x='monetary_value', y='frequency', ax=axs[2])
axs[2].set_title('Monetary vs. Frequency')

plt.tight_layout(pad=2)
plt.show()

#### 1.1.1. Customer Level

In [ ]:
customer_level = old_c_txn.groupby('customer_level').agg({
    'rfm_class': lambda x: len(x)
})
customer_level.rename(columns={'rfm_class': 'count'}, inplace=True)

plt.figure(figsize = (8, 6))
sns.barplot(x='count',
            y=customer_level.index,
            data=customer_level, palette='tab10',
            order=customer_level.sort_values('count', ascending=False).index)

plt.title('Customer Level')
plt.xlabel('Count')
plt.ylabel('Customer Level')
plt.show()

In [ ]:
bins = [0, 19, 29, 39, 49, 59, 69, old_c_txn['age'].max()]
labels = ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '>70']
old_c_txn['age_class'] = pd.cut(old_c_txn['age'], bins=bins, labels=labels)

plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='age_class', hue='customer_level', palette='tab10')
plt.title('Customer Level by Age')
plt.xlim(0, 650)
plt.legend(title='Level', loc='upper right')
plt.show()

#### 1.1.2. Gender

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='age_class', hue='gender', palette='tab10')
plt.title('Gender by Age')
plt.xlim(0, 650)
plt.legend(title='Gender', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='wealth_segment', hue='gender', palette='tab10')
plt.title('Gender by Wealth Segment')
plt.legend(title='Gender', loc='lower right')
plt.show()

#### 1.1.3. Wealth Segment

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='age_class', hue='wealth_segment', palette='tab10')
plt.title('Wealth Segment by Age')
plt.xlim(0, 650)
plt.legend(title='Wealth Segment', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='owns_car', hue='wealth_segment', palette='tab10')
plt.title('Wealth Segment by Car Ownership')
plt.xlim(0, 899)
plt.legend(title='Wealth Segment', loc='lower right')
plt.show()

#### 1.1.4. Car Ownership

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='job_industry_category', hue='owns_car', palette='tab10')
plt.title('Car Ownership by Job Industry')
plt.legend(['Not Own a Car', 'Owns Car'], loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='state', hue='owns_car', palette='tab10')
plt.title('Car Ownership by State')
plt.legend(['Not Own a Car', 'Owns Car'], loc='lower right')
plt.show()

#### 1.1.5. State

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='age_class', hue='state', palette='tab10')
plt.title('State by Age')
plt.xlim(0, 650)
plt.legend(title='State', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=old_c_txn, y='customer_level', hue='state', palette='tab10',
              order=['Platinum', 'Gold', 'Silver', 'Classic'])
plt.title('State by Customer Level')
plt.legend(title='State', loc='upper right')
plt.show()

#### 1.1.6. Job Industry

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=old_c_txn, y='job_industry_category', hue='m_score', palette='tab10')
plt.title('Job Industries with M Score')
plt.legend(title='M Score')
plt.show()

### 1.2. RFM Segmentation

Source: [RFM Segmentation](https://documentation.bloomreach.com/engagement/docs/rfm-segmentation)

In [ ]:
oc = old_c_txn.groupby('customer_level').agg({'age': lambda x: len(x)})
oc.rename(columns={'age':'count'}, inplace=True)
oc['count'] = oc['count'].astype(int)
oc.T

In [ ]:
print(f'Total Old Customer: {oc.sum().values}')

In [ ]:
oc = old_c_txn.groupby(['state','customer_level']).agg({'age': lambda x: len(x)})
oc.rename(columns={'age':'count'}, inplace=True)
oc['count'] = oc['count'].astype(int)
oc.T

In [ ]:
oc = old_c_txn.groupby(['total_score']).agg({'age': lambda x: x.count()}).cumsum()
oc.rename(columns={'age':'count'},inplace=True)
oc['count'] = oc['count'].astype(int)
oc.T

#### 1.2.0. Customer Segment

In [ ]:
old_c_txn['customer_segment'] = old_c_txn['total_score'].map({
    3: 'Champions',
    4: 'Loyalists',
    5: 'Potential Loyalists',
    6: 'New Customers',
    7: 'Promising',
    8: 'Need Attention',
    9: 'About to Sleep',
    10: 'High Risk',
    11: 'Hibernating',
    12: 'Lost Customers'
})
old_c_txn['customer_segment'].value_counts()

In [ ]:
old_c_txn.head()

#### 1.2.1. Top 1000 Old Customer

In [ ]:
top_1000_old_customer = old_c_txn.sort_values('rfm_class').head(1000)
top_1000_old_customer.head()

## 2. NewCustomerList Dataset

In [ ]:
new_c.columns

In [ ]:
new_c = new_c.drop(['first_name', 'last_name',
                    'past_3_years_bike_related_purchases',
                    'DOB', 'job_title', 'deceased_indicator',
                    'tenure', 'address', 'postcode', 'country',
                    'property_valuation', 'Rank', 'Value'], axis=1)

In [ ]:
new_column_order = ['customer_id', 'age', 'gender', 'job_industry_category',
                    'wealth_segment', 'owns_car', 'state']

# Reorder column
new_c = new_c.reindex(columns=old_column_order)
new_c.columns

In [ ]:
new_c.shape

In [ ]:
new_c.head()

### 2.1. Data Insights

#### 2.1. Gender

In [ ]:
bins = [0, 19, 29, 39, 49, 59, 69, new_c['age'].max()]
labels = ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '>70']
new_c['age_class'] = pd.cut(new_c['age'], bins=bins, labels=labels)

plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='age_class', hue='gender', palette='tab10')
plt.title('Gender by Age')
plt.xlim(0, 119)
plt.legend(title='Gender', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='wealth_segment', hue='gender', palette='tab10')
plt.title('Gender by Wealth Segment')
plt.legend(title='Gender', loc='lower right')
plt.show()

#### 2.2. Wealth Segment

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='age_class', hue='wealth_segment', palette='tab10')
plt.title('Wealth Segment by Age')
plt.xlim(0, 119)
plt.legend(title='Wealth Segment', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='owns_car', hue='wealth_segment', palette='tab10')
plt.title('Wealth Segment by Car Ownership')
plt.xlim(0, 299)
plt.legend(title='Wealth Segment', loc='lower right')
plt.show()

#### 2.3. Car Ownership

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='job_industry_category', hue='owns_car', palette='tab10')
plt.title('Car Ownership by Job Industry')
plt.xlim(0, 119)
plt.legend(['Not Own a Car', 'Owns Car'], loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='state', hue='owns_car', palette='tab10')
plt.title('Car Ownership by State')
plt.legend(['Not Own a Car', 'Owns Car'], loc='lower right')
plt.show()

#### 2.4. State

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(data=new_c, y='age_class', hue='state', palette='tab10')
plt.title('State by Age')
plt.xlim(0, 119)
plt.legend(title='State', loc='upper right')
plt.show()

## 3. Export the Dataset

In [ ]:
# Export dataset to a xlsx file
old_c_txn.to_excel('OldCustomer_Transactions.xlsx', index=False)
top_1000_old_customer.to_excel('Top1000_OldCustomer.xlsx', index=False)